Proceso general
Limpieza de datos:
Se manejan valores nulos y se ajustan los tipos de datos.
Transformación de datos:
Se crean nuevas variables y se simplifican columnas existentes.
Eliminación de outliers:
Se filtran valores extremos en price.
Preparación para modelado:
Se convierten variables categóricas y el dataset final queda listo para análisis avanzados.


In [ ]:
#Importar librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Cargar el dataset
file_path = 'C://Users//ricar//Downloads//Vehicles.csv' 
df = pd.read_csv(file_path)

# **1. Manejo de valores nulos**
df['model_year'].fillna(df['model_year'].median(), inplace=True)
df['paint_color'].fillna('unknown', inplace=True)
df['is_4wd'].fillna(0, inplace=True)

# **2. Verificación y ajuste de tipos de datos**
df['date_posted'] = pd.to_datetime(df['date_posted'])
df['is_4wd'] = df['is_4wd'].astype(int)

# **3. Crear nuevas variables**
df['vehicle_age'] = df['date_posted'].dt.year - df['model_year']
df['condition_simplified'] = df['condition'].map({
    'excellent': 1,
    'like new': 1,
    'good': 0,
    'fair': 0,
    'salvage': 0
})

# **4. Detección de outliers**
q_low = df['price'].quantile(0.01)
q_high = df['price'].quantile(0.99)
df = df[(df['price'] >= q_low) & (df['price'] <= q_high)]


model_year: Los valores nulos se imputan con la mediana de la columna. Esto evita sesgos que podrían surgir al usar el promedio si hay valores extremos.
paint_color: Los valores nulos se rellenan con 'unknown', indicando la falta de información.
is_4wd: Los valores nulos se imputan con 0, asumiendo que NaN representa "No es tracción en las cuatro ruedas".

date_posted: Se convierte a un formato de fecha (datetime) para facilitar cálculos relacionados con el tiempo.
is_4wd: Se asegura que esta columna sea de tipo entero, lo que es importante para cálculos y modelado.

vehicle_age: Se calcula la antigüedad del vehículo como la diferencia entre el año de publicación del anuncio y el año del modelo. Esta variable es crucial para entender la relación entre la edad del vehículo y su precio.
condition_simplified: Se convierte la columna condition en una variable binaria para simplificar el análisis:
1: Excelente/Como nuevo.
0: Bueno/Regular.
Los outliers en el precio se eliminan utilizando el rango entre los percentiles 1% y 99%. Esto remueve valores extremadamente altos o bajos que podrían distorsionar los análisis y modelos.

Las columnas categóricas (type y paint_color) se transforman mediante one-hot encoding, creando nuevas columnas binarias para cada categoría.
drop_first=True elimina una categoría de referencia para evitar la multicolinealidad en los modelos.

# **5. Visualizaciones relevantes**

In [ ]:

# Configurar estilo de los gráficos
sns.set_theme(style="whitegrid")

# 1. Distribución de precios
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], kde=True, bins=30, color='blue')
plt.title('Distribución de Precios')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# 2. Relación entre edad del vehículo y precio
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='vehicle_age', y='price', hue='condition_simplified', palette='viridis', alpha=0.6)
plt.title('Relación entre Edad del Vehículo y Precio')
plt.xlabel('Edad del Vehículo (Años)')
plt.ylabel('Precio')
plt.legend(title='Condición Simplificada', loc='upper right')
plt.show()

In [ ]:
# 3. Distribución de colores de pintura
plt.figure(figsize=(10, 6))
sns.countplot(data=df, y='paint_color', order=df['paint_color'].value_counts().index, palette='muted')
plt.title('Distribución de Colores de Pintura')
plt.xlabel('Frecuencia')
plt.ylabel('Color de Pintura')
plt.show()

In [ ]:
# 4. Boxplot de precios por tipo de vehículo
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='type', y='price', palette='coolwarm')
plt.title('Precio por Tipo de Vehículo')
plt.xlabel('Tipo de Vehículo')
plt.ylabel('Precio')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 5. Matriz de correlación para variables numéricas
plt.figure(figsize=(10, 8))
corr_matrix = df[['price', 'vehicle_age', 'model_year', 'cylinders', 'days_listed']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
# **6. Preparar datos para modelado**
df = pd.get_dummies(df, columns=['type', 'paint_color'], drop_first=True)

In [ ]:
# Revisar datos finales
print(df.info())
print(df.head())

# Guardar el dataset procesado
df.to_csv('processed_data.csv', index=False)